<a href="https://colab.research.google.com/github/TaranehPaula/ETL/blob/main/Mini_Projeto_ETL_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas Instaladas

In [ ]:
!pip install pandas

In [ ]:
!pip install pandera

# Bibliotecas Importadas

In [ ]:
import pandas as pd

In [ ]:
import pandera as pa

# EXTRAÇÃO do DataSet

In [ ]:
df = pd.read_csv('ocorrencia_2010_2020.csv',sep=';',parse_dates=['ocorrencia_dia'], dayfirst = True)

In [ ]:
df.head(2)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0


## VALIDAÇÃO (Data Validation)

# Montagem do Schema do DataFrame com a Pandera

In [ ]:
schema = pa.DataFrameSchema(
    columns= {
        'codigo_ocorrencia':pa.Column(pa.Int),
        'codigo_ocorrencia2':pa.Column(pa.Int),
        'ocorrencia_classificacao':pa.Column(pa.String),
        'ocorrencia_cidade':pa.Column(pa.String),
        'ocorrencia_uf':pa.Column(pa.String, pa.Check.str_length(2,2),nullable=True),
        'ocorrencia_aerodromo':pa.Column(pa.String, nullable=True),
        'ocorrencia_dia':pa.Column(pa.DateTime),
        'ocorrencia_hora':pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        'total_recomendacoes':pa.Column(pa.Int)
    }
)

In [ ]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


In [ ]:
df.dtypes # para visualizar os tipos de dados que foram carregados

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [ ]:
'''
Já é possível observar uma inconsistência nas colunas 
'ocorrencia_dia' e 'ocorrencia_hora' 
onde os tipos que foi importado no dataframe é object
quando deveria ser date

PROBLEMA RESOLVIDO COM O PARÂMETRO: parse_dates=['ocorrencia_dia']

'''

"\nJá é possível observar uma inconsistência nas colunas \n'ocorrencia_dia' e 'ocorrencia_hora' \nonde os tipos que foi importado no dataframe é object\nquando deveria ser date\n\nPROBLEMA RESOLVIDO COM O PARÂMETRO: parse_dates=['ocorrencia_dia']\n\n"

## LIMPEZA (Data Cleaning)

- O Python está lendo as informações no formato correto?

- Será que existe alguma coluna completamente vazia?

- Será que existe alguma informação em alguma linha vazia?

In [ ]:
schema.validate(df) #Fazer a primeira conferência da validação do Schema

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


### Backup

In [ ]:
df2 = df.copy

### Dados inconsistentes encontrados para limpeza

In [ ]:
'''
ocorrencia_uf
**
ocorrencia_aerodromo
###!
####
****
*****
ocorrencia_hora
NULL

'''

'''
OBSERVAÇÃO SOBRE LIMPEZA DE DADOS NA / NAN / NAT ( que são os MISSING VALUES)

- NA (Not Avaliable ) você trata com PANDAS
- NAN (Not a Numeric) por se tratar de números você trata com  a biblioteca NUMPY
- NAT (Not a Time) por se tratar de data 

- Dado OUTLAIER: (geralmente ele é dropado para ser usado em um contexto futuro)
Os outliers são dados que se diferenciam drasticamente de todos os outros, ou seja, extrapolam a média.

ARTIGO IMPORTANTE SOBRE O TEMA: https://www.aquare.la/o-que-sao-outliers-e-como-trata-los-em-uma-analise-de-dados/

'''

'\nOBSERVAÇÃO SOBRE LIMPEZA DE DADOS NA / NAN / NAT ( que são os MISSING VALUES)\n\n- NA (Not Avaliable ) você trata com PANDAS\n- NAN (Not a Numeric) por se tratar de números você trata com  a biblioteca NUMPY\n- NAT (Not a Time) por se tratar de data \n\n- Dado OUTLAIER: (geralmente ele é dropado para ser usado em um contexto futuro)\nOs outliers são dados que se diferenciam drasticamente de todos os outros, ou seja, extrapolam a média.\n\nARTIGO IMPORTANTE SOBRE O TEMA: https://www.aquare.la/o-que-sao-outliers-e-como-trata-los-em-uma-analise-de-dados/\n\n'

In [ ]:
# Limpar todas as linhas que contenham '**'
#df.loc[df.ocorrencia_aerodromo == '**',['ocorrencia_aerodromo']] = pd.NA

In [ ]:
# limpar todas as inconsistências listadas acima com a função 'replace'

df.replace(['**','###!','####','****','*****','NULL'], pd.NA, inplace=True)

In [ ]:
df   

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


### Agora que eu já padronizei as minhas inconsistências com NA o python terá diversas funções para tratá-los um a um

In [ ]:
# Saber quantos <NA> eu tenho? Quantos dados não informados eu tenho?

df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [ ]:
df.isnull().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [ ]:
# Colocar um 'valor' no meu dado <NA> com a função fillna(valor, inplace=True)
df.fillna(0)

In [ ]:
# Colocar um valor em uma coluna específica que tenha um <NA> com a função fillna
# df.fillna(value={'total_recomendacoes':10}, inplace=True)

In [ ]:
# Excluir uma coluna / DROPAR que não faz sentido para a Análise
# importante: axis = 1 é coluna e axis = 0 é a linha

df.drop(['coluna1'], axis=1, inplace= True)


In [ ]:
# DROPAR /EXCLUIR TODOS OS VALORES <NA>
#df.dropna(inplace=True)

In [ ]:
# DROPAR /EXCLUIR O <NA> POR COLUNA
df.dropna(subset=['ocorrencia_uf'], inplace=True)

### Tratar valores duplicados

In [ ]:
# DROPAR VALORES DUPLICADOS
df.drop_duplicates(inplace=True)

# TRANSFORMAÇÃO (Data Transforming)

## Caso eu precise Alterar o índice do DataFrame

### Primeiro devemos verificar se a coluna a ser eleita indice é unica


#### *A partir do dados validados  e dos dados limpos,  nós poderemos manipular esses dados criando novos conjuntos de dados,gerando novas informações, que poderão ser utilizadas posteriormente*

In [ ]:
df.codigo_ocorrencia.is_unique

True

### informando ao python que o código da ocorrência vai passar a ser o indice do DataFrame

In [ ]:
#df.set_index('codigo_ocorrencia', inplace=True)  

### Resetando um índice alterado

In [ ]:
#df.reset_index(drop=True,inplace=True)

### Filtros

In [ ]:
df.loc[df.ocorrencia_uf.isnull()]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,<NA>,<NA>,2013-09-02,02:54:00,0


In [ ]:
df.loc[df.ocorrencia_aerodromo.isnull()]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,<NA>,2010-01-06,17:53:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,<NA>,2010-01-09,12:30:00,3
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,<NA>,2010-01-10,20:00:00,0
...,...,...,...,...,...,...,...,...,...
5741,79754,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,<NA>,2020-12-28,13:00:00,0
5744,79753,79753,INCIDENTE GRAVE,GOIATUBA,GO,<NA>,2020-12-29,12:00:00,0
5745,79755,79755,ACIDENTE,MATO RICO,PR,<NA>,2020-12-29,10:30:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0


In [ ]:
df.loc[df.ocorrencia_hora.isnull()]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
4100,78823,78823,INCIDENTE,CORUMBÁ,MS,<NA>,2017-07-09,NaN,0


In [ ]:
df.count()

codigo_ocorrencia           5752
codigo_ocorrencia2          5752
ocorrencia_classificacao    5752
ocorrencia_cidade           5752
ocorrencia_uf               5751
ocorrencia_aerodromo        3403
ocorrencia_dia              5752
ocorrencia_hora             5751
total_recomendacoes         5752
dtype: int64

In [ ]:
# Quero saber as ocorrencias com mais de 10 recomendações

filtro_recomendacoes = df.total_recomendacoes >10
df.loc[filtro_recomendacoes]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
63,39992,39992,ACIDENTE,SÃO PAULO,SP,SBMT,2010-02-17,20:07:00,11
326,42250,42250,ACIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-08-12,12:26:00,19
530,43471,43471,ACIDENTE,SÃO PAULO,SP,<NA>,2010-12-14,20:15:00,11
727,44377,44377,ACIDENTE,ITÁPOLIS,SP,SDIO,2011-04-09,14:50:00,11
872,44796,44796,ACIDENTE,RECIFE,PE,SBRF,2011-07-13,09:54:00,23
884,44888,44888,INCIDENTE GRAVE,SÃO JOSÉ DOS PINHAIS,PR,SBCT,2011-07-15,17:05:00,11
1062,45554,45554,INCIDENTE GRAVE,UBERLÂNDIA,MG,SBUL,2011-11-15,02:15:00,20
1972,47938,47938,INCIDENTE,BRASÍLIA,DF,SBBR,2013-04-13,18:00:00,11
2795,52265,52265,ACIDENTE,SANTOS,SP,<NA>,2014-08-13,13:03:00,13
3930,66432,66432,INCIDENTE GRAVE,VITÓRIA,ES,<NA>,2017-02-21,11:47:00,12


In [ ]:
# Quero saber as cidades com as ocorrencias com mais de 10 recomendações 
filtro_recomendacoes = df.total_recomendacoes >10
df.loc[filtro_recomendacoes,'ocorrencia_cidade']

63                 SÃO PAULO
326           RIO DE JANEIRO
530                SÃO PAULO
727                 ITÁPOLIS
872                   RECIFE
884     SÃO JOSÉ DOS PINHAIS
1062              UBERLÂNDIA
1972                BRASÍLIA
2795                  SANTOS
3930                 VITÓRIA
Name: ocorrencia_cidade, dtype: object

In [ ]:
# Quero saber as cidades com as ocorrencias com mais de 10 recomendações e o total de recomendações em cada cidade
filtro_recomendacoes = df.total_recomendacoes >10
df.loc[filtro_recomendacoes,['ocorrencia_cidade','total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
63,SÃO PAULO,11
326,RIO DE JANEIRO,19
530,SÃO PAULO,11
727,ITÁPOLIS,11
872,RECIFE,23
884,SÃO JOSÉ DOS PINHAIS,11
1062,UBERLÂNDIA,20
1972,BRASÍLIA,11
2795,SANTOS,13
3930,VITÓRIA,12


In [ ]:
# Filtrar as ocorrências do tipo Grave
filtro_graves = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro_graves]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,<NA>,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,<NA>,2010-01-10,20:00:00,0
29,39709,39709,INCIDENTE GRAVE,CURITIBA,PR,SBBI,2010-01-23,16:36:00,2
35,39487,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
...,...,...,...,...,...,...,...,...,...
5741,79754,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,<NA>,2020-12-28,13:00:00,0
5744,79753,79753,INCIDENTE GRAVE,GOIATUBA,GO,<NA>,2020-12-29,12:00:00,0
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0


In [ ]:
# Filtrar as ocorrências do tipo Grave no Estado de São Paulo
filtro_graves = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro_SP = df.ocorrencia_uf == 'SP'
df.loc[filtro_graves & filtro_SP]   # Usamos o operador & (e) para juntar os dois

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
136,40270,40270,INCIDENTE GRAVE,PIRASSUNUNGA,SP,SDPY,2010-04-15,20:15:00,0
171,40620,40620,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2010-05-06,15:50:00,0
219,41411,41411,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-06-08,19:30:00,2
297,42289,42289,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-07-29,20:06:00,0
314,42323,42323,INCIDENTE GRAVE,SÃO JOSÉ DO RIO PRETO,SP,SBSR,2010-08-05,12:01:00,0
...,...,...,...,...,...,...,...,...,...
5591,79540,79540,INCIDENTE GRAVE,SÃO PAULO,SP,<NA>,2020-09-12,13:10:00,0
5610,79566,79566,INCIDENTE GRAVE,REGENTE FEIJÓ,SP,SDYJ,2020-09-27,19:33:00,0
5704,79739,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5705,79705,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0


In [ ]:
# Filtrar as ocorrências do tipo Incidente Grave ou Incidente no Estado de São Paulo
filtro_graves = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE') # usamos o opoerador | 'ou'
filtro_SP = df.ocorrencia_uf == 'SP'
df.loc[filtro_graves & filtro_SP]   # Usamos o operador & para juntar os dois

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,39809,INCIDENTE,SÃO PAULO,SP,<NA>,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,<NA>,2010-01-15,17:45:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,<NA>,2010-01-20,13:10:00,0
27,39768,39768,INCIDENTE,CAMPINAS,SP,<NA>,2010-01-21,20:45:00,0
33,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...,...
5704,79739,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5705,79705,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0
5715,79718,79718,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2020-12-11,13:50:00,0
5742,79800,79800,INCIDENTE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00,0


In [ ]:
# 2ª forma de Filtrar as ocorrências do tipo Incidente Grave ou Incidente no Estado de São Paulo, usando a função ''.isin'
filtro_graves = df.ocorrencia_classificacao.isin == (['INCIDENTE GRAVE', 'INCIDENTE']) 
filtro_SP = df.ocorrencia_uf == 'SP'
df.loc[filtro_graves & filtro_SP]   # Usamos o operador & para juntar os dois


,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes


In [ ]:
# pesquisar ocorrências do ano de 2015
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro_ocorrencias]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2996,52992,52992,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2015-01-01,11:40:00,3
2997,52979,52979,ACIDENTE,IVINHEMA,MS,<NA>,2015-01-02,14:50:00,0
2998,53073,53073,INCIDENTE,TEFÉ,AM,SBTF,2015-01-02,16:28:00,0
2999,53074,53074,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-01-02,20:34:00,0
3000,52976,52976,ACIDENTE,TOLEDO,PR,SBTD,2015-01-04,22:04:00,6
...,...,...,...,...,...,...,...,...,...
3462,60632,60632,INCIDENTE GRAVE,ITABERÁ,SP,<NA>,2015-12-24,14:00:00,0
3463,60600,60600,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-25,19:00:00,0
3464,60642,60642,INCIDENTE,SÃO FRANCISCO DO SUL,SC,SSSS,2015-12-26,16:00:00,0
3465,60631,60631,ACIDENTE,MAÇAMBARÁ,RS,<NA>,2015-12-28,19:00:00,2


In [ ]:
# pesquisar ocorrências do ano de 2015 e mês 12
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia.dt.month == 12
df.loc[filtro_ocorrencias & filtro_ocorrencias_mes]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3428,53573,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
3429,60601,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
3430,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3431,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
3432,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3433,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3434,53625,53625,ACIDENTE,TRINDADE,GO,<NA>,2015-12-06,13:10:00,3
3435,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3436,53628,53628,ACIDENTE,AGUAÍ,SP,<NA>,2015-12-08,14:30:00,1
3437,53629,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [ ]:
# pesquisar ocorrências do ano de 2015 e mês 12 e dia 8
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia.dt.month == 12
filtro_ocorrencias_dia  = df.ocorrencia_dia.dt.day == 8
df.loc[filtro_ocorrencias & filtro_ocorrencias_mes & filtro_ocorrencias_dia]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3436,53628,53628,ACIDENTE,AGUAÍ,SP,<NA>,2015-12-08,14:30:00,1,2015-12-08 14:30:00
3437,53629,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
3438,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0,2015-12-08 16:19:00
3439,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


In [ ]:
# pesquisar ocorrências do ano de 2015 e mês 12 e do dia 3 ao dia 8
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia.dt.month == 12
filtro_ocorrencias_dia_inicio  = df.ocorrencia_dia.dt.day >2 
filtro_ocorrencias_dia_fim = df.ocorrencia_dia.dt.day <9
df.loc[filtro_ocorrencias & filtro_ocorrencias_mes & filtro_ocorrencias_dia_inicio & filtro_ocorrencias_dia_fim]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3432,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3433,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3434,53625,53625,ACIDENTE,TRINDADE,GO,<NA>,2015-12-06,13:10:00,3
3435,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3436,53628,53628,ACIDENTE,AGUAÍ,SP,<NA>,2015-12-08,14:30:00,1
3437,53629,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3438,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3439,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


*Para poder fazer um filtro onde eu possa obter informações do ano, mês, dia e hora de determinada ocorrência a melhor opção é criar uma nova coluna onde eu junto dia e hora, no entando algumas funções foram adiconadas (pd.to_datetime) para converter tudo no formato date e (.astype(str)) para conseguir concatenar a data e a hora.

In [ ]:
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' / ' + df.ocorrencia_hora)

In [ ]:
df.head(2)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00


In [ ]:
# pesquisar ocorrências do ano de 2015 e mês 12 e do dia 3 ao dia 8 mostrando dia e hora
filtro_ocorrencias = df.ocorrencia_dia_hora.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia_hora.dt.month == 12
filtro_ocorrencias_dia_inicio  = df.ocorrencia_dia_hora.dt.day >2 
filtro_ocorrencias_dia_fim = df.ocorrencia_dia_hora.dt.day <9
df.loc[filtro_ocorrencias & filtro_ocorrencias_mes & filtro_ocorrencias_dia_inicio & filtro_ocorrencias_dia_fim]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3432,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0,2015-12-03 10:50:00
3433,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
3434,53625,53625,ACIDENTE,TRINDADE,GO,<NA>,2015-12-06,13:10:00,3,2015-12-06 13:10:00
3435,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
3436,53628,53628,ACIDENTE,AGUAÍ,SP,<NA>,2015-12-08,14:30:00,1,2015-12-08 14:30:00
3437,53629,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
3438,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0,2015-12-08 16:19:00
3439,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


## Group By  (Data Aggregating)

### *Agrupamentos de dados para gerar novas informações*

In [ ]:
# Grupo de ocorrências do ano de 2015 e mês 03
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro_ocorrencias & filtro_ocorrencias_mes]
df201503

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3086,53183,53183,ACIDENTE,AMAPORÃ,PR,<NA>,2015-03-02,23:00:00,0,2015-03-02 23:00:00
3087,53120,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
3088,53109,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
3089,53112,53112,ACIDENTE,MOGI GUAÇU,SP,<NA>,2015-03-06,21:00:00,0,2015-03-06 21:00:00
3090,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
3091,53167,53167,INCIDENTE,MARABÁ,PA,SBMA,2015-03-10,17:33:00,0,2015-03-10 17:33:00
3092,53596,53596,INCIDENTE,ITAPEMA,SC,<NA>,2015-03-10,21:35:00,0,2015-03-10 21:35:00
3093,53149,53149,ACIDENTE,TABATINGA,AM,SBTT,2015-03-11,23:25:00,0,2015-03-11 23:25:00
3094,53148,53148,ACIDENTE,ARARAS,SP,SDEH,2015-03-12,12:40:00,0,2015-03-12 12:40:00
3095,53153,53153,INCIDENTE,PORTO ALEGRE,RS,SBPA,2015-03-13,15:45:00,0,2015-03-13 15:45:00


In [ ]:
df201503.count()

codigo_ocorrencia           37
codigo_ocorrencia2          37
ocorrencia_classificacao    37
ocorrencia_cidade           37
ocorrencia_uf               37
ocorrencia_aerodromo        21
ocorrencia_dia              37
ocorrencia_hora             37
total_recomendacoes         37
ocorrencia_dia_hora         37
dtype: int64

In [ ]:
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

In [ ]:
df201503.groupby(['ocorrencia_classificacao']).size()  #contar os registros agrupados

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
dtype: int64

In [ ]:
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count().sort_values()

ocorrencia_classificacao
INCIDENTE GRAVE     5
ACIDENTE           15
INCIDENTE          17
Name: codigo_ocorrencia, dtype: int64

In [ ]:
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE          17
ACIDENTE           15
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

### Agrupamento dos dados das ocorrências na região Sudeste no ano de 2010

In [ ]:
filtro_ocorrencias_ano = df.ocorrencia_dia.dt.year == 2010
filtro_ocorrencias_uf = df.ocorrencia_uf.isin(['SP','MG','ES','RJ'])
dfSudeste2010 = df.loc[filtro_ocorrencias_ano & filtro_ocorrencias_uf]
dfSudeste2010

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0,2010-01-03 12:00:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,<NA>,2010-01-10,20:00:00,0,2010-01-10 20:00:00
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,2010-01-10 14:50:00
16,39809,39809,INCIDENTE,SÃO PAULO,SP,<NA>,2010-01-15,15:00:00,0,2010-01-15 15:00:00
...,...,...,...,...,...,...,...,...,...,...
541,43722,43722,ACIDENTE,CAJOBI,SP,<NA>,2010-12-23,11:00:00,0,2010-12-23 11:00:00
544,43548,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0,2010-12-26 22:10:00
548,43724,43724,ACIDENTE,COSMÓPOLIS,SP,<NA>,2010-12-29,12:00:00,0,2010-12-29 12:00:00
550,43730,43730,INCIDENTE,VITÓRIA,ES,<NA>,2010-12-30,18:10:00,0,2010-12-30 18:10:00


In [ ]:
# Quantos incidentes, acidentes e incidentes graves eu tenho na Região Sudeste?

dfSudeste2010.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE            40
INCIDENTE          170
INCIDENTE GRAVE     18
dtype: int64

In [ ]:
# Quantos incidentes, acidentes e incidentes graves eu tenho POR ESATDO (UF) na Região Sudeste?
dfSudeste2010.groupby(['ocorrencia_classificacao','ocorrencia_uf']).size()

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  MG                7
                          RJ                9
                          SP               24
INCIDENTE                 ES                5
                          MG               39
                          RJ               52
                          SP               74
INCIDENTE GRAVE           MG                5
                          RJ                4
                          SP                9
dtype: int64

In [ ]:
# Quantos incidentes, acidentes e incidentes graves eu tenho POR CIDADE na Região Sudeste?
dfSudeste2010.groupby(['ocorrencia_classificacao','ocorrencia_cidade']).size().sort_values(ascending=False)

ocorrencia_classificacao  ocorrencia_cidade
INCIDENTE                 RIO DE JANEIRO       38
                          SÃO PAULO            22
                          GUARULHOS            17
                          BELO HORIZONTE       14
                          CAMPINAS             13
                                               ..
                          ARRAIAL DO CABO       1
                          BEBEDOURO             1
ACIDENTE                  ARAÇATUBA             1
INCIDENTE                 DIAMANTINA            1
INCIDENTE GRAVE           TIMÓTEO               1
Length: 73, dtype: int64

In [ ]:
# Qual foi o total de recomendações por aerodromo do Rio de Janeiro?

filtro_RJ = dfSudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
dfSudeste2010.loc[filtro_RJ].total_recomendacoes.sum()

25

In [ ]:
# Qual foi o total de recomendações por cidades da Região Sudeste?

dfSudeste2010.groupby(['ocorrencia_cidade']).total_recomendacoes.sum()


ocorrencia_cidade
AMERICANA                 0
ARARAQUARA                0
ARAÇATUBA                 0
AREALVA                   0
ARRAIAL DO CABO           0
BAURU                     0
BEBEDOURO                 0
BELO HORIZONTE            2
BOCAINA                   0
BOITUVA                   1
BOM JESUS DO AMPARO       0
BOM JESUS DO GALHO        1
BRAGANÇA PAULISTA         9
BROTAS                    0
CAJOBI                    0
CAMANDUCAIA               9
CAMPINAS                  0
CAMPOS DOS GOYTACAZES     0
CASIMIRO DE ABREU         0
CONFINS                   0
COROMANDEL                0
COSMÓPOLIS                0
DIAMANTINA                0
DIVINÓPOLIS               0
GUARULHOS                 0
IPATINGA                  0
IPERÓ                     3
ITANHAÉM                  1
ITÁPOLIS                  0
JUIZ DE FORA              0
JUNDIAÍ                   7
LINHARES                  0
MACAÉ                     0
MANGARATIBA               0
MARÍLIA                   0
MO

In [ ]:
# Total de ocorrencias POR AERODROMO na Região Sudeste

dfSudeste2010.groupby(['ocorrencia_aerodromo'], dropna=False).total_recomendacoes.sum()

ocorrencia_aerodromo
SBAE     0
SBAQ     0
SBBH     0
SBBP     9
SBBU     0
SBCF     0
SBGL     3
SBGR     0
SBIP     0
SBJD     7
SBJR     3
SBKP     0
SBME     0
SBMK     0
SBML     0
SBMT    27
SBPC     0
SBPR     1
SBRJ    19
SBRP     0
SBSJ     0
SBSP     0
SBSR     0
SBUL     0
SBUR     0
SBVT     0
SDAM     0
SDOI     1
SDPW     0
SDPY     0
SNDT     0
SNDV     0
SNXT     0
NaN     45
Name: total_recomendacoes, dtype: int64

  # Considerações Finais

In [ ]:
# Aqui você pode colocar o resumo de como você trabalhou o ETL para que o Cientista ou Analista saiba exatamente o que você fez em resumo.

# LOAD (Data Loading)